# Network Metrics

Build the network graph, compute centrality metrics, detect communities, and save outputs to `data/processed/`

## Prep
Run preprocessing and temporal preprocessing if processed files are missing

In [ ]:
# Uncomment to regenerate processed data
# !python3 ../src/preprocessing.py
# !python3 ../src/temporal_preprocessing.py

## Setup
Import helpers / ensure the project `src` directory is on the path

In [1]:
import os, sys
# Resolve project root as parent of this notebook's directory
NOTEBOOK_DIR = os.path.abspath(os.getcwd())
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir))
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from build_network import load_and_build
from centrality_analysis import compute_centrality_measures, save_centrality_rankings
from community_detection import detect_communities, save_communities

print(f"Project root: {PROJECT_ROOT}")

Project root: /Users/masoncacurak/Downloads/CS_5483/dallas_network_analysis


## Build graph
Select a time period weight (e.g., `AM`, `Midday`, `PM`, `Evening`) or leave `period=None` to use `weight_type` (`congested`/`freeflow`).

In [ ]:
period = "Midday" # options: "AM", "Midday", "PM", "Evening", or None
G = load_and_build(weight_type="congested", period=period) # if period=None, weight_type="congested" or "freeflow"
print(f"Graph built: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

Loading processed_nodes.csv...
Loading processed_links.csv...
Building graph using temporal weight column 'travel_time_Midday'
Adding 21389 nodes...
Adding 35696 edges...
Graph build complete: 21389 nodes, 35696 edges
Graph built: 21389 nodes, 35696 edges
Loading processed_nodes.csv...
Loading processed_links.csv...
Building graph using congested_time as weight
Adding 21389 nodes...
Adding 35696 edges...
Graph build complete: 21389 nodes, 35696 edges
Graph built: 21389 nodes, 35696 edges


## Centrality metrics
Compute degree, betweenness (weighted), and eigenvector centrality --> save rankings to CSV

In [ ]:
centrality = compute_centrality_measures(G)
centrality_path = save_centrality_rankings(centrality)
print(f"Centrality rankings saved to: {centrality_path}")

## Community detection
Run Louvain on the undirected graph (Set `run_gn_sample=True` to also run Girvan–Newman on sampled subgraph)

In [ ]:
partition, num_comms, sizes = detect_communities(G, run_gn_sample=True)
communities_path = save_communities(partition)

print(f"Communities saved to: {communities_path}")
print(f"Total communities: {num_comms}")
print(f"Top community sizes: {sizes[:10]}")